In [ ]:
from   wrfpre   import wrfcase 
from   wrfpre   import wps
from   gribviz  import gribdata
from   datetime import datetime, timedelta
import os

In [ ]:
# construct a wps object from the namelist file
input = wps().plot_domains()

In [ ]:
# define simulation period 
day_range   = [1, 15]
month_range = [1, 1]
year        = 2022
spinup_days = 1

# set workpackage info 
vki_base_dir = '/zfs60/nobackup/stipa/WRF-Test-Py'
vki_project  = 'C4W-WRF'
vki_date     = '26-11-2024'
vki_empl     = 'Stipa'
job_forcing  = 'ERA5'

In [ ]:
# create base folder (contains all WRF cases)
os.makedirs(vki_base_dir, exist_ok=True)

# create WRF cases
wrf_cases     = []
sim_start     = datetime(year, month_range[0], day_range[0])
sim_end       = datetime(year, month_range[1], day_range[1])
cur_job_start = sim_start
act_job_start = cur_job_start - timedelta(days=spinup_days)

while cur_job_start < sim_end:
    
    cur_job_end = cur_job_start + timedelta(days=7)

    if cur_job_end > sim_end:
        cur_job_end = sim_end

    job_dir     =  act_job_start.strftime("%Y_%m_%d") + "-" + cur_job_end.strftime("%Y_%m_%d")
    job_name    = vki_project + "_" + vki_date + "_" + vki_empl + "_" + job_forcing + "_" + job_dir
    
    # initialize wrf case class list 
    wrf_cases.append(wrfcase(vki_project,
                              vki_date,
                              job_name,
                              vki_empl,
                              job_forcing,
                              job_dir,
                              vki_base_dir))

    print("Creating case       : ", job_name, " in ", wrf_cases[-1].case_root)
    print("   w/o spinup       : from ", cur_job_start, " to ", cur_job_end)
    print("   actual w/ spinup : from ", act_job_start, " to ", cur_job_end)
    print("   spinup           : ", spinup_days, " days")
    
    # create directories and copy files 
    wrf_cases[-1].case_init('/nobackup/users/stipa/software/wrf', 
                            '/nobackup/users/stipa/software/wps')
    
    # create pressure level dataset 
    data_pl = gribdata(day_range    = [act_job_start.day, cur_job_end.day], 
                    month_range     = [act_job_start.month, cur_job_end.month],
                    year_range      = [act_job_start.year, cur_job_end.year], 
                    prefix          = "levels", 
                    extension       = "grib", 
                    path            = "/storage/wrf/nobackup/munters/ERA5"
                    )

    # create single levels dataset
    data_sl = gribdata(day_range    = [act_job_start.day, cur_job_end.day], 
                    month_range     = [act_job_start.month, cur_job_end.month], 
                    year_range      = [act_job_start.year, cur_job_end.year], 
                    prefix          = "single", 
                    extension       = "grib", 
                    path            = "/storage/wrf/nobackup/munters/ERA5"
                    )
    
    data_pl.link_files_to_dir(wrf_cases[-1].data_dir) 
    data_sl.link_files_to_dir(wrf_cases[-1].data_dir)

    # create namelist files
    wrf_cases[-1].create_namelist_wps(act_job_start.year, act_job_start.month, act_job_start.day, 0,
                             cur_job_end.year, cur_job_end.month, cur_job_end.day, 0)
    
    wrf_cases[-1].create_namelist_wrf([act_job_start.year, act_job_start.year, act_job_start.year],
                             [act_job_start.month, act_job_start.month, act_job_start.month], 
                             [act_job_start.day,   act_job_start.day,   act_job_start.day], 
                             [0, 0, 0],
                             [cur_job_end.year,   cur_job_end.year,   cur_job_end.year], 
                             [cur_job_end.month,   cur_job_end.month,   cur_job_end.month], 
                             [cur_job_end.day,     cur_job_end.day,     cur_job_end.day],  
                             [0, 0, 0])
    
    # activate restart for all but the first case
    if cur_job_start > sim_start:
        wrf_cases[-1].set_entry('namelist.input', 'time_control', 'restart', '.true.,')


    # create job script
    wrf_cases[-1].wps_setup()

    cur_job_start = cur_job_end 
    act_job_start = cur_job_start - timedelta(days=spinup_days)
